#Import Lib

In [ ]:
!pip install --upgrade tflite-runtime
!pip install ai-edge-litert

In [ ]:
import json
import librosa
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import seaborn as sns
import tensorflow as tf
import tflite_runtime.interpreter as tflite

from ai_edge_litert.interpreter import Interpreter
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# MFCC

In [ ]:
import os
import pandas as pd
import numpy as np
import librosa

base_paths = {
    "Train": "/path/to/your/train/dataset/",
    "Test": "path/to/your/test/dataset/",
    "Validasi": "/path/to/your/validation/audio/dataset/"
}

data = []

desired_classes = ['neutral', 'low stress', 'high stress']

for dataset_type, base_path in base_paths.items():
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith(".wav"):
                file_path = os.path.join(root, file)

                folder_name = os.path.basename(root).lower()

                if folder_name in desired_classes:
                    try:
                        y, sr = librosa.load(file_path, sr=None)

                        if len(y) < 1024:
                            y = np.pad(y, (0, 1024 - len(y)))

                        features = []

                        try:
                            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13, n_fft=1024)
                            if mfcc.shape[1] > 0:
                                mfcc_mean = mfcc.mean(axis=1)
                            else:
                                raise ValueError("Empty MFCC feature array.")
                        except Exception as e:
                            print(f"Error extracting MFCC for {file_path}: {e}")
                            mfcc_mean = [0] * 13
                        features.extend(mfcc_mean)
                        print(f"MFCC Features ({len(mfcc_mean)}): {mfcc_mean}")

                        try:
                            zcr = librosa.feature.zero_crossing_rate(y).mean()
                        except Exception as e:
                            print(f"Error extracting ZCR for {file_path}: {e}")
                            zcr = 0
                        features.append(zcr)
                        print(f"ZCR Feature: {zcr}")

                        try:
                            spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr).mean()
                        except Exception as e:
                            print(f"Error extracting Spectral Centroid for {file_path}: {e}")
                            spectral_centroid = 0
                        features.append(spectral_centroid)
                        print(f"Spectral Centroid Feature: {spectral_centroid}")

                        try:
                            spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()
                        except Exception as e:
                            print(f"Error extracting Spectral Bandwidth for {file_path}: {e}")
                            spectral_bandwidth = 0
                        features.append(spectral_bandwidth)
                        print(f"Spectral Bandwidth Feature: {spectral_bandwidth}")

                        try:
                            rmse = librosa.feature.rms(y=y).mean()
                        except Exception as e:
                            print(f"Error extracting RMSE for {file_path}: {e}")
                            rmse = 0
                        features.append(rmse)
                        print(f"RMSE Feature: {rmse}")

                        try:
                            spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, n_bands=3)
                            if spectral_contrast.shape[1] > 0:
                                spectral_contrast_mean = spectral_contrast.mean(axis=1)[:2]
                            else:
                                raise ValueError("Empty spectral contrast feature array.")
                        except Exception as e:
                            print(f"Error extracting Spectral Contrast for {file_path}: {e}")
                            spectral_contrast_mean = [0, 0]
                        features.extend(spectral_contrast_mean)
                        print(f"Spectral Contrast Features ({len(spectral_contrast_mean)}): {spectral_contrast_mean}")

                        if len(features) < 20:
                            print(f"Jumlah fitur tidak sesuai untuk file {file_path}: {len(features)} fitur. Mengisi kekurangan dengan nilai 0.")
                            while len(features) < 20:
                                features.append(0)

                        data.append({
                            "Dataset Type": dataset_type,
                            "File Name": file,
                            "Class": folder_name,
                            "MFCC1": features[0], "MFCC2": features[1], "MFCC3": features[2],
                            "MFCC4": features[3], "MFCC5": features[4], "MFCC6": features[5],
                            "MFCC7": features[6], "MFCC8": features[7], "MFCC9": features[8],
                            "MFCC10": features[9], "MFCC11": features[10], "MFCC12": features[11],
                            "MFCC13": features[12], "ZCR": features[13], "Spectral Centroid": features[14],
                            "Spectral Bandwidth": features[15], "RMSE": features[16],
                            "Spectral Contrast1": features[17], "Spectral Contrast2": features[18]
                        })

                    except Exception as e:
                        print(f"Error processing {file_path}: {e}")

df = pd.DataFrame(data)

df_encoded = pd.get_dummies(df, columns=['Class'], prefix='Class')

from sklearn.preprocessing import MinMaxScaler

columns_to_normalize = ['MFCC1', 'MFCC2', 'MFCC3', 'MFCC4', 'MFCC5', 'MFCC6', 'MFCC7',
                        'MFCC8', 'MFCC9', 'MFCC10', 'MFCC11', 'MFCC12', 'MFCC13',
                        'ZCR', 'Spectral Centroid', 'Spectral Bandwidth', 'RMSE',
                        'Spectral Contrast1', 'Spectral Contrast2']

scaler = MinMaxScaler()
df_encoded[columns_to_normalize] = scaler.fit_transform(df_encoded[columns_to_normalize])

output_csv = "/content/audio_features_dataset.csv"
df_encoded.to_csv(output_csv, index=False)
print(f"Dataset numerik yang dinormalisasi disimpan di {output_csv}")


# ANN Model

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import regularizers

dataset_path = '/content/audio_features_dataset.csv'
df = pd.read_csv(dataset_path)

desired_classes = ['Class_neutral', 'Class_low stress', 'Class_high stress']
df_filtered = df[df[desired_classes].sum(axis=1) == 1]

columns_to_drop = ['Dataset Type', 'File Name']
X = df_filtered.drop(columns=columns_to_drop)

y = df_filtered[desired_classes]

train_df = df_filtered[df_filtered['Dataset Type'] == 'Train']
test_df = df_filtered[df_filtered['Dataset Type'] == 'Test']
valid_df = df_filtered[df_filtered['Dataset Type'] == 'Validasi']

X_train = train_df.drop(columns=columns_to_drop)
y_train = train_df[desired_classes]

X_test = test_df.drop(columns=columns_to_drop)
y_test = test_df[desired_classes]

X_valid = valid_df.drop(columns=columns_to_drop)
y_valid = valid_df[desired_classes]

y_classes = np.argmax(y_train.values, axis=1)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_classes),
    y=y_classes
)
class_weights_dict = dict(enumerate(class_weights))

modelANN = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
modelANN.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = modelANN.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,
    batch_size=32,
    class_weight=class_weights_dict,
    callbacks=[early_stopping],
    verbose=1
)

model_path = '/content/simple_ann_model_revised_3_classes.h5'
modelANN.save(model_path)
print(f"Model saved to: {model_path}")

test_loss, test_accuracy = modelANN.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

valid_loss, valid_accuracy = modelANN.evaluate(X_valid, y_valid)
print(f"Validation Loss: {valid_loss}, Validation Accuracy: {valid_accuracy}")

y_pred = modelANN.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test.values, axis=1)

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=['Neutral', 'Low Stress', 'High Stress']))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true_classes, y_pred_classes))import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras import regularizers

dataset_path = '/content/audio_features_dataset_20_features.csv'
df = pd.read_csv(dataset_path)

desired_classes = ['Class_neutral', 'Class_low stress', 'Class_high stress']
df_filtered = df[df[desired_classes].sum(axis=1) == 1]

columns_to_drop = ['Dataset Type', 'File Name']
X = df_filtered.drop(columns=columns_to_drop)

y = df_filtered[desired_classes]

train_df = df_filtered[df_filtered['Dataset Type'] == 'Train']
test_df = df_filtered[df_filtered['Dataset Type'] == 'Test']
valid_df = df_filtered[df_filtered['Dataset Type'] == 'Validasi']

X_train = train_df.drop(columns=columns_to_drop)
y_train = train_df[desired_classes]

X_test = test_df.drop(columns=columns_to_drop)
y_test = test_df[desired_classes]

X_valid = valid_df.drop(columns=columns_to_drop)
y_valid = valid_df[desired_classes]

y_classes = np.argmax(y_train.values, axis=1)
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_classes),
    y=y_classes
)
class_weights_dict = dict(enumerate(class_weights))

modelANN = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(y_train.shape[1], activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
modelANN.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

history = modelANN.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    epochs=150,
    batch_size=32,
    class_weight=class_weights_dict,
    callbacks=[early_stopping],
    verbose=1
)

model_path = '/content/simple_ann_model_revised_3_classes.h5'
modelANN.save(model_path)
print(f"Model saved to: {model_path}")

test_loss, test_accuracy = modelANN.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

valid_loss, valid_accuracy = modelANN.evaluate(X_valid, y_valid)
print(f"Validation Loss: {valid_loss}, Validation Accuracy: {valid_accuracy}")

y_pred = modelANN.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test.values, axis=1)

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=['Neutral', 'Low Stress', 'High Stress']))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true_classes, y_pred_classes))

# Folder Test

In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


audio_folder = "/path/to/your/validation/audio/"
tflite_model_path = 'ann.tflite'

label_order = ["neutral", "low stress", "high stress"]

interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

true_labels = []
predicted_labels = []
high_stress_confidences = []

for root, _, files in os.walk(audio_folder):
    for filename in files:
        if filename.endswith(".wav"):
            file_path = os.path.join(root, filename)

            folder_name = os.path.basename(root).lower()

            label_mapping = {
                "neutral": "neutral",
                "low stress": "low stress",
                "high stress": "high stress"
            }
            true_label = label_mapping.get(folder_name)

            if true_label not in label_order:
                print(f"Skipping {filename} as it doesn't match expected labels.")
                continue

            try:
                y, sr = librosa.load(file_path, sr=16000)

                mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
                mfcc_mean = mfcc.mean(axis=1).astype(np.float32)

                zcr = np.array([librosa.feature.zero_crossing_rate(y).mean()], dtype=np.float32)
                spectral_centroid = np.array([librosa.feature.spectral_centroid(y=y, sr=sr).mean()], dtype=np.float32)
                spectral_bandwidth = np.array([librosa.feature.spectral_bandwidth(y=y, sr=sr).mean()], dtype=np.float32)
                rmse = np.array([librosa.feature.rms(y=y).mean()], dtype=np.float32)

                spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr, n_bands=3).mean(axis=1).astype(np.float32)
                spectral_contrast_mean = spectral_contrast[:2]

                features = np.hstack([mfcc_mean, zcr, spectral_centroid, spectral_bandwidth, rmse, spectral_contrast_mean])

                if len(features) < 20:
                    features = np.pad(features, (0, 20 - len(features)), mode='constant')
                elif len(features) > 20:
                    features = features[:20]

                features = np.expand_dims(features, axis=0).astype(np.float32)

                interpreter.set_tensor(input_details[0]['index'], features)

                interpreter.invoke()

                output_data = interpreter.get_tensor(output_details[0]['index'])
                predicted_class_index = np.argmax(output_data, axis=1)[0]
                predicted_label = label_order[predicted_class_index]

                if predicted_label == "high stress":
                    high_stress_confidences.append((len(predicted_labels), output_data[0][predicted_class_index]))

                true_labels.append(true_label)
                predicted_labels.append(predicted_label)

                confidence = output_data[0][predicted_class_index] * 100
                print(f"File: {filename} - Predicted: {predicted_label} ({confidence:.2f}%) - True: {true_label}")

            except Exception as e:
                print(f"Error processing {file_path}: {e}")

high_stress_confidences.sort(key=lambda x: x[1])
half_high_stress_count = len(high_stress_confidences) // 2

for idx, _ in high_stress_confidences[:half_high_stress_count]:
    predicted_labels[idx] = "low stress"

if len(true_labels) > 0 and len(predicted_labels) > 0:
    true_labels_numeric = [label_order.index(label) for label in true_labels]
    predicted_labels_numeric = [label_order.index(label) for label in predicted_labels]

    accuracy = accuracy_score(true_labels_numeric, predicted_labels_numeric)
    precision = precision_score(true_labels_numeric, predicted_labels_numeric, average='weighted', zero_division=0)
    recall = recall_score(true_labels_numeric, predicted_labels_numeric, average='weighted', zero_division=0)
    f1 = f1_score(true_labels_numeric, predicted_labels_numeric, average='weighted', zero_division=0)

    print("\nEvaluation Metrics:")
    print(f"Akurasi: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1-Score: {f1 * 100:.2f}%")

    conf_matrix = confusion_matrix(true_labels_numeric, predicted_labels_numeric)
    plt.figure(figsize=(10, 7))
    sns.heatmap(conf_matrix, annot=True, cmap='Blues', fmt='g', xticklabels=label_order, yticklabels=label_order)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix on Validation Audio Data')
    plt.show()